In [1]:
# 1. Get the data
# 2. Split the data into train and test
# 3. Train an LDA on all the questions from the training set

# 4. Accept a question from for which we need to find the set of users
# 5. Get questions asked by this user (User-QA)
# 6. Get Twitter/HP/About-me data about the user (User-non-QA)
# 7. Calculate p(q|theta_qa) from User-QA
# 8. Calculate p(q|theta_non_qa) from User-non-QA
# 9. Calculate Equation (2)
# 10. Calculate Equation (1)

# 11. Rank the users

# 12. From the test set, get the set of users that have answered the question
# 13. Evaluate the result

# 1. Get the data

In [2]:
#Import Libraries
import xml.etree.ElementTree
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re
import matplotlib.pyplot as plt
import math 
import time
from nltk import FreqDist


#Get root tag of XML
e = xml.etree.ElementTree.parse('../data/android.meta.stackexchange.com.backup/Posts.xml').getroot()
u = xml.etree.ElementTree.parse('../data/android.meta.stackexchange.com.backup/Users.xml').getroot()

stop_words = set(stopwords.words('english'))

In [3]:
#Functions

def remove_html_tags(text):
    if text==None:
        return ""
    text=text.encode('utf-8')
    p=re.compile('<[^>]*>|\n')
    return p.sub(" ", text)
    
def clean_tags(text):
    if text==None:
        return ""
    text=text.encode('utf-8')
    p=re.compile('<|>')
    return p.sub(" ", text)

def get_alpha(x):
    x = 0.25 * (abs(x)-5)
    return 1 / (1 + math.exp(-x))

def get_posts(user_id):
    rows = e.findall('.//row[@OwnerUserId="'+ user_id +'"][@PostTypeId="1"]')
    textual_content = ""
    
    for row in rows:
        content = remove_html_tags(row.get('Body'))
        content += remove_html_tags(row.get('Title'))
        content += clean_tags(row.get('Tags'))
        textual_content += content  
        
    return textual_content,get_alpha(len(rows))

def get_ground_truth(question_id):
    # Get the users who answered the question.
    rows = e.findall('.//row[@ParentId="'+ str(question_id) +'"][@PostTypeId="2"]')
    content = []
    for row in rows:
        if (row.get('OwnerUserId') != None):
            content.append(int(row.get('OwnerUserId')))
    return content

def get_epoch(t):
    pattern = '%Y-%m-%dT%H:%M:%S.%f'
    epoch = int(time.mktime(time.strptime(t, pattern)))
    return epoch
    
def get_prior(user_id, question_id):
    user_time  = last_access[user_id]
    que_time = time_of_question[question_id]
    return 1
#     return math.exp(-np.abs(que_time - user_time) / 86400) 

def get_frequency(text):
    return Counter(text.split()).most_common()


# 2. Split the data

In [4]:
questions = {}
ground_truths = {}
time_of_question = {}

#Get each question
for atype in e.findall('row'):
    id=atype.get('PostTypeId')
    if int(id)==1:
        body=atype.get('Body')
        title=atype.get('Title')
        tags=atype.get('Tags')
        que_id=int(atype.get('Id'))
        
        print('>>>>', que_id)
        ground_truths[que_id] = get_ground_truth(que_id)
        print('<<<<')
        
        body = remove_html_tags(body)
        title = remove_html_tags(title)
        tags=clean_tags(tags)
        post=str(body)+" "+str(title)+str(tags)

        questions[que_id] = post
        time_of_question[que_id] = get_epoch(atype.get('CreationDate'))
l=len(questions)

index=int(l*0.8)

#split
train_questions=dict(questions.items()[:index])
test_questions=dict(questions.items()[index:])
print(len(train_questions),len(test_questions))


#writing data to file
train_file_name = 'question_meta_test.txt'
# test_file_name = 'question_meta_train.txt'

questions = train_questions.values()
train_handler = open(train_file_name, 'w')
# test_handler = open(test_file_name, 'w')

train_list = []
# test_list = []
len_list = len(questions)
for i in range(len_list):
#     if i < 3*len_list/4:
    train_list.append(questions[i])
#     else:
#         test_list.append(questions[i])
        
train_handler.write("\n".join(train_list))
# test_handler.write("\n".join(test_list))
train_handler.close()
# test_handler.close()

print 'Done creating train and test data'

('>>>>', 1)
<<<<
('>>>>', 5)
<<<<
('>>>>', 6)
<<<<
('>>>>', 11)
<<<<
('>>>>', 17)
<<<<
('>>>>', 25)
<<<<
('>>>>', 26)
<<<<
('>>>>', 31)
<<<<
('>>>>', 32)
<<<<
('>>>>', 34)
<<<<
('>>>>', 37)
<<<<
('>>>>', 38)
<<<<
('>>>>', 43)
<<<<
('>>>>', 48)
<<<<
('>>>>', 53)
<<<<
('>>>>', 57)
<<<<
('>>>>', 59)
<<<<
('>>>>', 64)
<<<<
('>>>>', 66)
<<<<
('>>>>', 71)
<<<<
('>>>>', 76)
<<<<
('>>>>', 82)
<<<<
('>>>>', 89)
<<<<
('>>>>', 93)
<<<<
('>>>>', 96)
<<<<
('>>>>', 98)
<<<<
('>>>>', 99)
<<<<
('>>>>', 103)
<<<<
('>>>>', 111)
<<<<
('>>>>', 112)
<<<<
('>>>>', 114)
<<<<
('>>>>', 117)
<<<<
('>>>>', 125)
<<<<
('>>>>', 126)
<<<<
('>>>>', 128)
<<<<
('>>>>', 132)
<<<<
('>>>>', 135)
<<<<
('>>>>', 137)
<<<<
('>>>>', 140)
<<<<
('>>>>', 141)
<<<<
('>>>>', 148)
<<<<
('>>>>', 152)
<<<<
('>>>>', 159)
<<<<
('>>>>', 161)
<<<<
('>>>>', 164)
<<<<
('>>>>', 167)
<<<<
('>>>>', 168)
<<<<
('>>>>', 174)
<<<<
('>>>>', 176)
<<<<
('>>>>', 180)
<<<<
('>>>>', 182)
<<<<
('>>>>', 185)
<<<<
('>>>>', 189)
<<<<
('>>>>', 190)
<<<<
('>>

<<<<
('>>>>', 1352)
<<<<
('>>>>', 1354)
<<<<
('>>>>', 1356)
<<<<
('>>>>', 1359)
<<<<
('>>>>', 1361)
<<<<
('>>>>', 1362)
<<<<
('>>>>', 1364)
<<<<
('>>>>', 1366)
<<<<
('>>>>', 1368)
<<<<
('>>>>', 1372)
<<<<
('>>>>', 1377)
<<<<
('>>>>', 1379)
<<<<
('>>>>', 1381)
<<<<
('>>>>', 1383)
<<<<
('>>>>', 1384)
<<<<
('>>>>', 1385)
<<<<
('>>>>', 1386)
<<<<
('>>>>', 1388)
<<<<
('>>>>', 1391)
<<<<
('>>>>', 1392)
<<<<
('>>>>', 1393)
<<<<
('>>>>', 1394)
<<<<
('>>>>', 1398)
<<<<
('>>>>', 1400)
<<<<
('>>>>', 1402)
<<<<
('>>>>', 1404)
<<<<
('>>>>', 1405)
<<<<
('>>>>', 1407)
<<<<
('>>>>', 1408)
<<<<
('>>>>', 1410)
<<<<
('>>>>', 1414)
<<<<
('>>>>', 1416)
<<<<
('>>>>', 1417)
<<<<
('>>>>', 1418)
<<<<
('>>>>', 1420)
<<<<
('>>>>', 1428)
<<<<
('>>>>', 1429)
<<<<
('>>>>', 1431)
<<<<
('>>>>', 1433)
<<<<
('>>>>', 1435)
<<<<
('>>>>', 1440)
<<<<
('>>>>', 1442)
<<<<
('>>>>', 1446)
<<<<
('>>>>', 1450)
<<<<
('>>>>', 1452)
<<<<
('>>>>', 1457)
<<<<
('>>>>', 1459)
<<<<
('>>>>', 1460)
<<<<
('>>>>', 1461)
<<<<
('>>>>', 1463)


# Train LDA on Question profile

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from cStringIO import StringIO
import numpy as np
import time
import operator

In [6]:
collection = open(train_file_name)
n_top_words = 2000 
# Count term frequencies in the train data
tf_vectorizer = CountVectorizer(max_df = 5, min_df = 1, max_features = n_top_words, stop_words = 'english')
tf = tf_vectorizer.fit_transform(collection)
tf_feature_names = tf_vectorizer.get_feature_names()
n_topics = 20
# Fit LDA according to train data set
lda = LatentDirichletAllocation(n_topics = n_topics, max_iter = 5, doc_topic_prior = 0.1, topic_word_prior = 0.1, learning_method = 'online', learning_offset=50., random_state=0)
lda.fit(tf)



LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.1,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=0.1, total_samples=1000000.0, verbose=0)

In [7]:
int_nonqa = {} # key = userID, value = description  
ext_nonqa_twitter = {} # key = userID, value = description
ext_nonqa_hp = {} # key = userID, value = description

import re
#About
p=re.compile(r'&lt;(\S)*|p&gt;|\&#xA;|&quot;|/a&gt;|/li&gt;|/ul&gt;|/b&gt;|&gt;|;|#\d(\d)+|&amp')
a = open('user_internal_nonqa_info.txt')
for line in a:
    tmp=p.sub('', line) #remove all punctuations in i(single sentence) and store in tmp
    list_line = tmp.split('\t')
    int_nonqa[list_line[0]] = list_line[3]

#Twitter
a = open('user_external_nonqa_info.txt')
for line in a:
    list_line = line.split('\t')
    ext_nonqa_twitter[list_line[0]] = list_line[1]

#HomePage
a = open('user_external_nonqa_info_homepage_nouns.txt')
for line in a:
    list_line = line.split('\t')
    if(len(list_line) >= 2):
        ext_nonqa_hp[list_line[0]] = list_line[1]
        

In [8]:
def get_prob_q_given_u(question,userid):
    alpha=alpha_of_user[userid]
    qa=get_prob_q_given_u_qa(question,userid,'qa')
    non_qa=get_prob_q_given_u_qa(question,userid,'non-qa')
    return (alpha*qa+(1-alpha)*non_qa)


def get_theta_u(userid,t):
    data=""
    if t=='qa':  #For QA data 
        data = StringIO(LDA_UP[userid]) # single line file of all questions
    else:        #For NonQA data
        accountid=AccountId[userid]
        if ext_nonqa_hp.get(accountid) is not None:
            data += ext_nonqa_hp[accountid]
        if ext_nonqa_twitter.get(accountid) is not None:
            data += ext_nonqa_twitter[accountid]
        if int_nonqa.get(accountid) is not None:
            data += int_nonqa[accountid]
        
        data = re.sub(r"[\n\t\s]*", "", data)
        data=StringIO(data)
        
    tf_test = tf_vectorizer.transform(data)
    tf_test_features = tf_vectorizer.get_feature_names()
    
    try:
        pred = lda.transform(tf_test) # topic probabilities given QA profile
    except Exception, e:
        return
    
    list_features = [tf_test_features[j] for j in range(n_top_words)]
    feature_prob = {}
    topic_prob = []
    for topic_i, topic in enumerate(lda.components_): 
        feature_prob[topic_i] = [j for j in topic]
    if t=='qa':
        UP_QA_topic[int(user_id)],UP_QA_word[int(user_id)],UP_QA_feature[int(user_id)]=pred[0],feature_prob,list_features
    else:
        UP_NON_QA_topic[int(user_id)],UP_NON_QA_word[int(user_id)],UP_NON_QA_feature[int(user_id)]=pred[0],feature_prob,list_features

def get_prob_q_given_u_qa(question,userid,t):
    pred=[]
    feature_prob=[]
    list_features=[]
    if userid in UP_NON_QA_topic:
        pred=UP_NON_QA_topic[userid]
    else:
        return 0
    if userid in UP_NON_QA_word:
        feature_prob=UP_NON_QA_word[userid]
    if userid in UP_NON_QA_feature:
        list_features=UP_NON_QA_feature[userid]
    if t=='qa':  #For QA data 
        if userid in UP_QA_topic:
            pred=UP_QA_topic[userid]
        else:
            return 0
        if userid in UP_QA_word:
            feature_prob=UP_QA_word[userid]
        if userid in UP_QA_feature:
            list_features=UP_QA_feature[userid]
    eq5=0
    for word in question.split():
        word = word.lower()
        if word in list_features:
            word_i = list_features.index(word)
            for topic_i, topic_prob in enumerate(pred):
                eq5 += math.log(topic_prob) + math.log(feature_prob[topic_i][word_i])
        else:
            eq5 += 1e-10
    return eq5    

# User QA Data

In [9]:
LDA_UP={}
alpha_of_user={}
last_access={}
AccountId={}
UP_QA_topic={}
UP_QA_word={}
UP_QA_feature={}
UP_NON_QA_topic={}
UP_NON_QA_word={}
UP_NON_QA_feature={}
for users_row in u.findall('row'):
    user_id = users_row.get('Id')
#     print('UserId',user_id)
    LDA_UP[int(user_id)],alpha_of_user[int(user_id)] = get_posts(user_id)
    last_access[int(user_id)] = get_epoch(users_row.get('LastAccessDate'))
    AccountId[int(user_id)] = users_row.get('AccountId')
    print('Printing',user_id)
    print('Printing',user_id)
    get_theta_u(int(user_id),'qa')
    get_theta_u(int(user_id),'non_qa')
print len(LDA_UP)

('Printing', '-1')
('Printing', '-1')
('Printing', '1')
('Printing', '1')
('Printing', '2')
('Printing', '2')
('Printing', '3')
('Printing', '3')
('Printing', '5')
('Printing', '5')
('Printing', '7')
('Printing', '7')
('Printing', '9')
('Printing', '9')
('Printing', '10')
('Printing', '10')
('Printing', '12')
('Printing', '12')
('Printing', '13')
('Printing', '13')
('Printing', '14')
('Printing', '14')
('Printing', '15')
('Printing', '15')
('Printing', '17')
('Printing', '17')
('Printing', '19')
('Printing', '19')
('Printing', '20')
('Printing', '20')
('Printing', '21')
('Printing', '21')
('Printing', '23')
('Printing', '23')
('Printing', '25')
('Printing', '25')
('Printing', '26')
('Printing', '26')
('Printing', '27')
('Printing', '27')
('Printing', '30')
('Printing', '30')
('Printing', '33')
('Printing', '33')
('Printing', '35')
('Printing', '35')
('Printing', '36')
('Printing', '36')
('Printing', '38')
('Printing', '38')
('Printing', '39')
('Printing', '39')
('Printing', '41')
('Pri

('Printing', '552')
('Printing', '552')
('Printing', '558')
('Printing', '558')
('Printing', '563')
('Printing', '563')
('Printing', '564')
('Printing', '564')
('Printing', '578')
('Printing', '578')
('Printing', '584')
('Printing', '584')
('Printing', '594')
('Printing', '594')
('Printing', '596')
('Printing', '596')
('Printing', '600')
('Printing', '600')
('Printing', '602')
('Printing', '602')
('Printing', '618')
('Printing', '618')
('Printing', '621')
('Printing', '621')
('Printing', '622')
('Printing', '622')
('Printing', '623')
('Printing', '623')
('Printing', '624')
('Printing', '624')
('Printing', '625')
('Printing', '625')
('Printing', '628')
('Printing', '628')
('Printing', '630')
('Printing', '630')
('Printing', '639')
('Printing', '639')
('Printing', '641')
('Printing', '641')
('Printing', '649')
('Printing', '649')
('Printing', '656')
('Printing', '656')
('Printing', '660')
('Printing', '660')
('Printing', '668')
('Printing', '668')
('Printing', '670')
('Printing', '670')


('Printing', '1518')
('Printing', '1518')
('Printing', '1522')
('Printing', '1522')
('Printing', '1524')
('Printing', '1524')
('Printing', '1535')
('Printing', '1535')
('Printing', '1541')
('Printing', '1541')
('Printing', '1542')
('Printing', '1542')
('Printing', '1548')
('Printing', '1548')
('Printing', '1551')
('Printing', '1551')
('Printing', '1558')
('Printing', '1558')
('Printing', '1562')
('Printing', '1562')
('Printing', '1572')
('Printing', '1572')
('Printing', '1575')
('Printing', '1575')
('Printing', '1576')
('Printing', '1576')
('Printing', '1597')
('Printing', '1597')
('Printing', '1601')
('Printing', '1601')
('Printing', '1608')
('Printing', '1608')
('Printing', '1610')
('Printing', '1610')
('Printing', '1619')
('Printing', '1619')
('Printing', '1633')
('Printing', '1633')
('Printing', '1634')
('Printing', '1634')
('Printing', '1640')
('Printing', '1640')
('Printing', '1645')
('Printing', '1645')
('Printing', '1656')
('Printing', '1656')
('Printing', '1659')
('Printing', 

('Printing', '3091')
('Printing', '3091')
('Printing', '3098')
('Printing', '3098')
('Printing', '3100')
('Printing', '3100')
('Printing', '3116')
('Printing', '3116')
('Printing', '3117')
('Printing', '3117')
('Printing', '3122')
('Printing', '3122')
('Printing', '3125')
('Printing', '3125')
('Printing', '3134')
('Printing', '3134')
('Printing', '3138')
('Printing', '3138')
('Printing', '3141')
('Printing', '3141')
('Printing', '3142')
('Printing', '3142')
('Printing', '3151')
('Printing', '3151')
('Printing', '3156')
('Printing', '3156')
('Printing', '3163')
('Printing', '3163')
('Printing', '3166')
('Printing', '3166')
('Printing', '3175')
('Printing', '3175')
('Printing', '3182')
('Printing', '3182')
('Printing', '3183')
('Printing', '3183')
('Printing', '3190')
('Printing', '3190')
('Printing', '3192')
('Printing', '3192')
('Printing', '3202')
('Printing', '3202')
('Printing', '3203')
('Printing', '3203')
('Printing', '3205')
('Printing', '3205')
('Printing', '3207')
('Printing', 

('Printing', '5013')
('Printing', '5013')
('Printing', '5047')
('Printing', '5047')
('Printing', '5082')
('Printing', '5082')
('Printing', '5090')
('Printing', '5090')
('Printing', '5092')
('Printing', '5092')
('Printing', '5094')
('Printing', '5094')
('Printing', '5099')
('Printing', '5099')
('Printing', '5112')
('Printing', '5112')
('Printing', '5128')
('Printing', '5128')
('Printing', '5131')
('Printing', '5131')
('Printing', '5132')
('Printing', '5132')
('Printing', '5156')
('Printing', '5156')
('Printing', '5194')
('Printing', '5194')
('Printing', '5208')
('Printing', '5208')
('Printing', '5212')
('Printing', '5212')
('Printing', '5213')
('Printing', '5213')
('Printing', '5224')
('Printing', '5224')
('Printing', '5232')
('Printing', '5232')
('Printing', '5234')
('Printing', '5234')
('Printing', '5242')
('Printing', '5242')
('Printing', '5244')
('Printing', '5244')
('Printing', '5251')
('Printing', '5251')
('Printing', '5275')
('Printing', '5275')
('Printing', '5285')
('Printing', 

...

('Printing', '215477')
('Printing', '215477')
('Printing', '215504')
('Printing', '215504')
('Printing', '215529')
('Printing', '215529')
('Printing', '215532')
('Printing', '215532')
('Printing', '215565')
('Printing', '215565')
('Printing', '215585')
('Printing', '215585')
('Printing', '215627')
('Printing', '215627')
('Printing', '215655')
('Printing', '215655')
('Printing', '215670')
('Printing', '215670')
('Printing', '215673')
('Printing', '215673')
('Printing', '215680')
('Printing', '215680')
('Printing', '215686')
('Printing', '215686')
('Printing', '215691')
('Printing', '215691')
('Printing', '215693')
('Printing', '215693')
('Printing', '215716')
('Printing', '215716')
('Printing', '215810')
('Printing', '215810')
('Printing', '215887')
('Printing', '215887')
('Printing', '215912')
('Printing', '215912')
('Printing', '216156')
('Printing', '216156')
('Printing', '216279')
('Printing', '216279')
('Printing', '216287')
('Printing', '216287')
('Printing', '216290')
('Printing'

('Printing', '222657')
('Printing', '222657')
('Printing', '222708')
('Printing', '222708')
('Printing', '222727')
('Printing', '222727')
('Printing', '222731')
('Printing', '222731')
('Printing', '222813')
('Printing', '222813')
('Printing', '222860')
('Printing', '222860')
('Printing', '222873')
('Printing', '222873')
('Printing', '222897')
('Printing', '222897')
('Printing', '222901')
('Printing', '222901')
('Printing', '222938')
('Printing', '222938')
('Printing', '223023')
('Printing', '223023')
('Printing', '223077')
('Printing', '223077')
('Printing', '223130')
('Printing', '223130')
('Printing', '223159')
('Printing', '223159')
('Printing', '223170')
('Printing', '223170')
('Printing', '223308')
('Printing', '223308')
('Printing', '223380')
('Printing', '223380')
('Printing', '223414')
('Printing', '223414')
('Printing', '223522')
('Printing', '223522')
('Printing', '223524')
('Printing', '223524')
('Printing', '223532')
('Printing', '223532')
('Printing', '223537')
('Printing'

('Printing', '231383')
('Printing', '231383')
('Printing', '231410')
('Printing', '231410')
('Printing', '231440')
('Printing', '231440')
('Printing', '231471')
('Printing', '231471')
('Printing', '231481')
('Printing', '231481')
('Printing', '231504')
('Printing', '231504')
('Printing', '231536')
('Printing', '231536')
('Printing', '231572')
('Printing', '231572')
('Printing', '231585')
('Printing', '231585')
('Printing', '231588')
('Printing', '231588')
('Printing', '231647')
('Printing', '231647')
('Printing', '231671')
('Printing', '231671')
('Printing', '231686')
('Printing', '231686')
('Printing', '231716')
('Printing', '231716')
('Printing', '231763')
('Printing', '231763')
('Printing', '231865')
('Printing', '231865')
('Printing', '231876')
('Printing', '231876')
('Printing', '231889')
('Printing', '231889')
('Printing', '231904')
('Printing', '231904')
('Printing', '231914')
('Printing', '231914')
('Printing', '231956')
('Printing', '231956')
('Printing', '231959')
('Printing'

('Printing', '236015')
('Printing', '236015')
('Printing', '236043')
('Printing', '236043')
('Printing', '236044')
('Printing', '236044')
('Printing', '236109')
('Printing', '236109')
('Printing', '236158')
('Printing', '236158')
('Printing', '236161')
('Printing', '236161')
('Printing', '236202')
('Printing', '236202')
('Printing', '236278')
('Printing', '236278')
('Printing', '236292')
('Printing', '236292')
('Printing', '236311')
('Printing', '236311')
('Printing', '236337')
('Printing', '236337')
('Printing', '236363')
('Printing', '236363')
('Printing', '236376')
('Printing', '236376')
('Printing', '236379')
('Printing', '236379')
('Printing', '236422')
('Printing', '236422')
('Printing', '236423')
('Printing', '236423')
('Printing', '236430')
('Printing', '236430')
('Printing', '236433')
('Printing', '236433')
('Printing', '236449')
('Printing', '236449')
('Printing', '236463')
('Printing', '236463')
('Printing', '236539')
('Printing', '236539')
('Printing', '236577')
('Printing'

('Printing', '242542')
('Printing', '242542')
('Printing', '242552')
('Printing', '242552')
('Printing', '242684')
('Printing', '242684')
('Printing', '242694')
('Printing', '242694')
('Printing', '242727')
('Printing', '242727')
('Printing', '242787')
('Printing', '242787')
('Printing', '242802')
('Printing', '242802')
('Printing', '242827')
('Printing', '242827')
('Printing', '242861')
('Printing', '242861')
('Printing', '242871')
('Printing', '242871')
('Printing', '242880')
('Printing', '242880')
('Printing', '242894')
('Printing', '242894')
('Printing', '243168')
('Printing', '243168')
('Printing', '243230')
('Printing', '243230')
('Printing', '243306')
('Printing', '243306')
('Printing', '243307')
('Printing', '243307')
('Printing', '243320')
('Printing', '243320')
('Printing', '243399')
('Printing', '243399')
('Printing', '243416')
('Printing', '243416')
('Printing', '243479')
('Printing', '243479')
('Printing', '243600')
('Printing', '243600')
('Printing', '243601')
('Printing'

In [10]:
# print(LDA_UP)
print(AccountId)

{1: '2', 2: '34933', 3: '18098', 229380: '11382494', 5: '48534', 7: '74184', 9: '27837', 10: '317', 12: '526173', 13: '26217', 14: '106983', 15: '174853', 17: '116139', 19: '3579', 20: '28900', 21: '21729', 23: '86536', 25: '1998', 26: '163375', 27: '60544', 30: '9496', 33: '14430', 35: '445', 36: '4380', 38: '20321', 39: '29738', 41: '16723', 43: '3354', 45: '108712', 46: '11234', 48: '3213', 49: '47754', 50: '22390', 51: '43726', 53: '23949', 55: '2193', 56: '162686', 57: '39943', 32827: '947589', 60: '41923', 61: '13250', 62: '212239', 63: '185392', 64: '32541', 65: '544298', 66: '131601', 67: '18673', 68: '28424', 69: '1618', 70: '91373', 71: '40051', 75: '21232', 32845: '2467529', 79: '58693', 80: '157369', 81: '8833', 82: '30755', 84: '21721', 85: '1', 86: '54253', 32856: '207775', 32861: '2683509', 94: '2449', 95: '27021', 96: '1305', 98: '10331', 100: '16313', 102: '17239', 196625: '8397947', 104: '20010', 105: '76959', 106: '24377', 107: '42562', 110: '107594', 111: '15104', 1

# Non QA Data

# Evaluation

In [11]:
def calculate_MRR_per_qn(pred, actual):
    for i, p in enumerate(pred):
        if(p in actual):
            return (1./(i+1))
    return 0

def calculate_precision_per_qn(pred, actual):
    n_common_items = len(set(pred) - (set(pred) - set(actual)))
    return (n_common_items * 1.) / len(pred)


In [12]:
mrr_s = []
precision_s = []
user_id_question_map = {}
for que_id in test_questions.keys():
    user_list = []
    user_prob = []
    for userid in LDA_UP.keys():
        prob = get_prob_q_given_u(test_questions[que_id],userid) + math.log(get_prior(userid, que_id))
        user_list.append(userid)
        user_prob.append(prob)
    ids = np.array(user_prob).argsort()[:10]
    userids = np.array(user_list)[ids]
    complete_ids = np.array(user_prob).argsort()
    complete_userids = np.array(user_list)
    user_id_question_map[que_id] = complete_userids
    
    mrr_s.append(calculate_MRR_per_qn(userids, ground_truths[que_id]))
    precision_s.append(calculate_precision_per_qn(userids, ground_truths[que_id]))
    
#     print ('que_id, ids, groud_truth, userids ', que_id, ids,  ground_truths[que_id],userids)
    print ('(current)Mean Reciprocal Rank: ', np.mean(mrr_s))
    print ('(current)Precision @ 10', np.mean(precision_s))
    print ('\n')
#     break
    
mrr = np.mean(mrr_s)
prec = np.mean(precision_s)

print ('(final)Mean Reciprocal Rank: ', mrr)
print ('(final)Precision @ 10', prec)

('(current)Mean Reciprocal Rank: ', 0.1111111111111111)
('(current)Precision @ 10', 0.1)


('(current)Mean Reciprocal Rank: ', 0.2222222222222222)
('(current)Precision @ 10', 0.1)


('(current)Mean Reciprocal Rank: ', 0.14814814814814814)
('(current)Precision @ 10', 0.06666666666666667)


('(current)Mean Reciprocal Rank: ', 0.1111111111111111)
('(current)Precision @ 10', 0.05)


('(current)Mean Reciprocal Rank: ', 0.11111111111111112)
('(current)Precision @ 10', 0.06000000000000001)


('(current)Mean Reciprocal Rank: ', 0.0925925925925926)
('(current)Precision @ 10', 0.05000000000000001)


('(current)Mean Reciprocal Rank: ', 0.07936507936507937)
('(current)Precision @ 10', 0.042857142857142864)


('(current)Mean Reciprocal Rank: ', 0.08333333333333333)
('(current)Precision @ 10', 0.05)


('(current)Mean Reciprocal Rank: ', 0.07407407407407407)
('(current)Precision @ 10', 0.044444444444444446)


('(current)Mean Reciprocal Rank: ', 0.06666666666666667)
('(current)Precision @ 10', 0.04)



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:66: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal



('(current)Mean Reciprocal Rank: ', 0.07820512820512819)
('(current)Precision @ 10', 0.03538461538461539)


('(current)Mean Reciprocal Rank: ', 0.077020202020202)
('(current)Precision @ 10', 0.03484848484848485)


('(current)Mean Reciprocal Rank: ', 0.08084577114427859)
('(current)Precision @ 10', 0.03582089552238806)


('(current)Mean Reciprocal Rank: ', 0.07965686274509802)
('(current)Precision @ 10', 0.03529411764705883)


('(current)Mean Reciprocal Rank: ', 0.07850241545893717)
('(current)Precision @ 10', 0.03478260869565218)


('(current)Mean Reciprocal Rank: ', 0.07738095238095236)
('(current)Precision @ 10', 0.03428571428571429)


('(current)Mean Reciprocal Rank: ', 0.07981220657276993)
('(current)Precision @ 10', 0.03521126760563381)


('(current)Mean Reciprocal Rank: ', 0.08217592592592592)
('(current)Precision @ 10', 0.03611111111111112)


('(current)Mean Reciprocal Rank: ', 0.08105022831050228)
('(current)Precision @ 10', 0.03561643835616439)


('(current)Mean Reciprocal Ra


('(current)Mean Reciprocal Rank: ', 0.09448356807511736)
('(current)Precision @ 10', 0.035211267605633804)


('(current)Mean Reciprocal Rank: ', 0.09382284382284381)
('(current)Precision @ 10', 0.03496503496503497)


('(current)Mean Reciprocal Rank: ', 0.0931712962962963)
('(current)Precision @ 10', 0.03472222222222223)


('(current)Mean Reciprocal Rank: ', 0.09482758620689655)
('(current)Precision @ 10', 0.03517241379310346)


('(current)Mean Reciprocal Rank: ', 0.09417808219178082)
('(current)Precision @ 10', 0.03493150684931508)


('(current)Mean Reciprocal Rank: ', 0.0935374149659864)
('(current)Precision @ 10', 0.034693877551020415)


('(current)Mean Reciprocal Rank: ', 0.0929054054054054)
('(current)Precision @ 10', 0.03445945945945947)


('(current)Mean Reciprocal Rank: ', 0.09228187919463088)
('(current)Precision @ 10', 0.034228187919463096)


('(current)Mean Reciprocal Rank: ', 0.09166666666666666)
('(current)Precision @ 10', 0.03400000000000001)


('(current)Mean Reciprocal 

In [13]:
import pickle
pickle_file = 'user_id_question_map.pkl'
pickle.dump(user_id_question_map, open(pickle_file, 'wb'))

In [ ]:
print user_id_question_map.shape

AttributeError: 'dict' object has no attribute 'shape'